In [2]:
# Déterminer le gène d'intérêt
gene = 'ENSG00000012048'

In [3]:
##### ÉTAPE 1: IMPORTER LES PRÉDICTIONS DE FOMONET #####

import pickle # Importation du module

with open(f'../data/{gene}_orfs.pkl', 'rb') as f: # Ouverture du fichier contenant les prédictions de FOMOnet sous forme de dictionnaire
    orf = pickle.load(f)
print(len(orf)) # Impression du dictionnaire des prédictions

32014


In [4]:
##### ÉTAPE 2: FAIRE LE GENE_DIC ET LE CDS_DIC DU GÈNE D'INTÉRÊT #####

# Le gene_dic contient tous les transcrits du gène. Il contient les positions de début et de fin de chaque exon faisant partie de chaque transcrit.

gene = 'ENSG00000012048'
import re # Importation du module pour les regular expression
gene_dic = {} # Création du dictionnaire vierge
gene_dic[gene]= {} # Ajoute le nom du gène (identifiant ENSG) comme 1ere clé et un sous-dictionnaire comme valeur
data = '../data/Homo_sapiens.GRCh38.113.gtf'
with open(data, 'r') as file: # Ouverture du GTF en mode read()
    for line in file: # Itération sur chaque ligne du GTF

        # Séparation dans les lignes à l'aide des caractères '\t' et '"'
        colonne_tab = line.split('\t') 
        colonne_guillemet = line.split('"')
            
        if gene in line: # Conserve les lignes qui portent sur le gène d'intérêt dans le GTF

            if colonne_tab[2]== 'gene': # Retient la ligne sur le gène général
                brand = colonne_tab[6] # Stock dans une variable la nature du brin +/-
                gene_name = colonne_guillemet[5] # Stock dans une variable le nom commun du gène. Ex. BRCA1

            if colonne_tab[2]== 'transcript': # Retient les lignes sur les transcrits
                transcrit_nom = colonne_guillemet[5] # Stock le nom du transcrit ENST dans une variable
                gene_dic[gene][transcrit_nom]= {} # Ajoute le nom des transcrits au 1er sous-dictionnaire (clé) et un sous-dictionnaire comme valeur

            if colonne_tab[2]== 'exon': # Retient les lignes d'exons
                transcrit_name_info = re.search(r'ENST[0123456789]{5,100}', line) # Recherche le nom du transcrit dans la ligne de l'exon
                transcrit_name = transcrit_name_info.group(0) # Stock le nom du transcrit ENST dans une variable
                exon_name_info = re.search(r'ENSE[0123456789]{5,100}', line) # Recherche le nom de l'exon dans la ligne de l'exon
                exon_name = exon_name_info.group(0) # Stock le nom de l'exon ENSE dans une variable
                petit = colonne_tab[3] # Position génomique la plus petite
                grand = colonne_tab[4] # Position génomique la plus grande

                for key in gene_dic[gene].keys(): # Itération sur chaque transcrit du gène
                    if key == transcrit_name: # La clé (transcrit) doit être le même que le nom du transcrit de la ligne
                        gene_dic[gene][transcrit_name][exon_name]= [] # Ajoute tous les exons appartenant au transcrit (clé) et une liste vide comme valeur

                for key in gene_dic[gene][transcrit_name].keys(): # Itération sur chaque exon présent dans un transcrit
                    if key == exon_name: # La clé (exon) doit être le même que le nom de l'exon de la ligne
                        gene_dic[gene][transcrit_name][exon_name].append(petit) # Ajoute la première position génomique dans la liste
                        gene_dic[gene][transcrit_name][exon_name].append(grand) # Ajoute la deuxième position génomique dans la liste

# Le cds_dic renferme les CDS pour chaque transcrit du gène. C'est-à-dire, les positions de début et de fin pour chaque exon en excluant les parties
# faisant partie du 5'UTR et du 3'UTR.

cds_dic = {} # Création du dictionnaire vierge
cds_dic[gene]= {} # Ajout du nom du gène comme 1ere clé avec un sous-dictionnaire comme valeur
data = '../data/Homo_sapiens.GRCh38.113.gtf'
with open(data,'r') as file: # Ouverture du GTF
    for line in file: # Itération sur chaque ligne du GTF
        # Séparation dans les lignes à l'aide des caractères '\t' et '"'
        colonne_tab = line.split('\t')
        colonne_guillemet = line.split('"')
        if len(colonne_tab) >= 3 and len(colonne_guillemet) >= 3: # Permet de retirer les premières lignes qui ne contiennent pas de données génomiques
            if gene in line: # Retient les lignes qui concerne le gène d'intérêt
                if colonne_tab[2]== 'transcript': # Retient les lignes de transcrit
                    transcrit = colonne_guillemet[5] # Stock le nom du transcrit ENST
                    cds_dic[gene][transcrit]= {} # Ajoute le nom du transcrit dans le 1er sous-dictionnaire et un sous-dictionnaire comme valeur
                if colonne_tab[2]== 'CDS': # Retient les lignes de CDS
                    transcrit_name_info = re.search(r'ENST[0123456789]{5,100}', line) # Recherche le nom du transcrit dans la ligne
                    transcrit_name = transcrit_name_info.group(0) # Stock le nom du transcrit dans une variable ENST
                    exon_number = colonne_guillemet[9] # Stock le numéro d'exon
                    petit = colonne_tab[3] # Position génomique la plus petite
                    grand = colonne_tab[4] # Position génomique la plus grande
                    for key in cds_dic[gene].keys(): # Itération sur chaque transcrit du gène
                        if key == transcrit_name: # La clé (transcrit) doit être égal au nom du transcrit dans la ligne
                            cds_dic[gene][transcrit_name][exon_number]= [] # Ajoute le numéro d'exon au 2e sous-dictionnaire et une liste comme valeur
                            cds_dic[gene][transcrit_name][exon_number].append(petit) # Ajoute la première position génomique dans la liste
                            cds_dic[gene][transcrit_name][exon_number].append(grand) # Ajoute la deuxième position génomique dans la liste

In [5]:
##### ÉTAPE 3: CRÉATION DES FONCTIONS SEQUENCE_MUTE(), SEQUENCE_CDNA() ET GET_POSITION_CHRONOLOGIQUE() #####

# La fonction sequence_mute permet de sortir la séquence d'ADN de n'importe quel transcrit ayant subit n'importe quelle mutation. 
def sequence_mute (transcrit, mutation_id): # Définir la fonction; elle dépend du transcrit et de la mutation_id
    header_of_transcrit = '' # Création d'une chaine qui contiendra le header recherché
    fichier = f"../result/pre_fomonet_{gene}/{gene}_all_transcripts_with_mutation.fasta" # Ouverture du fichier qui renferme toutes les séquences des transcrits mutés
    with open(fichier, 'r') as file: # Ouverture du fasta qui contient la séquence ADNc de tous les transcrits mutés du gènes d'intérêt
        for line in file: # Itération sur chaque ligne du fichier
            if gene in line and transcrit in line and mutation_id in line: # Retient la ligne avec le nom du gène, le nom du transcrit et la mutation id
                header_of_transcrit = line.strip() # Cette ligne devient le header_of_transcrit, car elle contient le gène et le transcrit recherché
                break # Quand le header est trouvé, on sort de la boucle

    if header_of_transcrit: # Si un header a été trouvé
        sequence = "" # Création d'une chaine qui contiendra la séquence ADNc du transcrit muté
        header_trouve = False

        with open(fichier, 'r') as file: # Ouverture du fichier fasta qui contient les séquences d'ADNc
            for line in file: # Itération sur chaque ligne du fichier
                line = line.strip()
                if line.startswith(">"): # Retient les lignes commençant par '>', donc les headers
                    if header_trouve:
                        break

                    if line == header_of_transcrit: # La ligne correspond au header recherché
                        header_trouve = True

                elif header_trouve:
                    sequence += line

        if sequence: # Si une séquence a été extraite
            return sequence
            
        else: # Si aucune n'a été trouvé
            return ("Séquence non trouvée.")
            
    else: # Si aucun header n'a été trouvé
        return (f"Le header {header_of_transcrit} n'a pas été trouvé.")


# La fonction sequence_cDNA() permet de sortir la séquence d'ADNc de n'importe quel transcrit ne contenant pas de mutation.
def sequence_cDNA (transcrit): # Définir la fonction; elle dépend seulement du transcrit
    header_of_transcrit = '' # Création d'une chaine qui contiendra le header recherché

    with open('../data/Homo_sapiens.GRCh38.cdna.all.fa', 'r') as file: # Ouverture du fasta qui contient la séquence ADNc de tous les transcrits des gènes Homo sapiens
        for line in file: # Itération sur chaque ligne du fichier
            if gene in line and transcrit in line: # Retient la ligne avec le nom du gène et le nom du transcrit
                header_of_transcrit = line.strip() # Cette ligne devient le header_of_transcrit, car elle contient le gène et le transcrit recherché
                break # Quand le header est trouvé, on sort de la boucle

    if header_of_transcrit: # Si un header a été trouvé
        sequence = "" # Création d'une chaine qui contiendra la séquence ADNc 
        header_trouve = False

        with open('../data/Homo_sapiens.GRCh38.cdna.all.fa', 'r') as file: # Ouverture du fichier fasta qui contient les séquences d'ADNc
            for line in file: # Itération sur chaque ligne du fichier
                line = line.strip()
                if line.startswith(">"): # Retient les lignes commençant par '>', donc les headers
                    if header_trouve:
                        break

                    if line == header_of_transcrit: # La ligne correspond au header recherché
                        header_trouve = True

                elif header_trouve:
                    sequence += line

        if sequence: # Si une séquence a été extraite
            return sequence
            
        else: # Si aucune n'a été trouvé
            return ("Séquence non trouvée.")
            
    else: # Si aucun header n'a été trouvé
        return (f"Le header {header_of_transcrit} n'a pas été trouvé.")
    

# La fonction get_position_chronologique() permet de sortir la position chronologique dans n'importe quel transcrit à partir de la position génomique.
def get_position_chronologique(position_chromosomique, transcrit): # Définir la fonction; elle dépend du transcrit et de la position chromosomique
    total = 0
    liste_position_fin_exon = [0] # Liste qui va contenir la position de fin de chaque exon en coordonnées chronologiques

    for exon in gene_dic[gene][transcrit]: # Itération sur chaque exon du transcrit
        longueur_exon = int(gene_dic[gene][transcrit][exon][1])-int(gene_dic[gene][transcrit][exon][0])+1 # Calcul de la longueur de l'exon en nombre de nucléotides
        total = total + int(longueur_exon) # Cette variable finit par contenir la longueur de l'exon et tous les exons précédents = position chronologique de fin pour chaque exon
        liste_position_fin_exon.append(total) # Ajoute les positions chronologiques de fin pour chaque exon

        # Conditions: la position chromosomique doit se retrouver obligatoire dans un exon du transcrit pour pouvoir être convertit en position chronologique
        if position_chromosomique >= int(gene_dic[gene][transcrit][exon][0]) and position_chromosomique <= int(gene_dic[gene][transcrit][exon][1]): 
            keys = list(gene_dic[gene][transcrit].keys()) # La variable keys renferme le nom de l'exon présentemment itéré, donc l'exon dans laquelle la position est incluse
            position_keys = keys.index(exon) # Je veux connaitre la position de l'exon dans le transcrit

            # Pour un gène sur le brin négatif
            if brand == '-':
            # Calcul de la conversion des coordonnées chromosomiques en coordonnées chronologiques dans un transcrit
                position_chronologique = int(liste_position_fin_exon[position_keys])+1+(int(gene_dic[gene][transcrit][exon][1])-position_chromosomique) 

            # Pour un gène sur le brin positif
            if brand == '+':
            # Calcul de la conversion des coordonnées chromosomiques en coordonnées chronologiques dans un transcrit
                position_chronologique = int(liste_position_fin_exon[position_keys])+1+(position_chromosomique-int(gene_dic[gene][transcrit][exon][0]))
    return position_chronologique # La fonction retourne la position chronologique dans le transcrit

In [6]:
##### ÉTAPE 4: CORRECTION DES POSITIONS DONNÉES PAR FOMONET #####

# Vérification que le nombre de données générés par FOMOnet est bon.
# La quantité de données devrait être égale à toutes les mutations pouvant être effectué sur chaque transcrit (en retirant les mutations qui sont erroné)
# et en ajoutant à cela, le nombre de transcrit existant pour ce gène.


# Retirer les mutations erronés
mutation_erronee = set()
with open(f'../result/pre_fomonet_{gene}/mutations_non_effectuees/{gene}_not_working.tsv', 'r') as file:
    for line in file:
        mutation_erronee.add(line)

# Compter chaque mutation pour chaque transcrit (en retirant les erroné)        
count_every_line = 0
for transcrit in gene_dic[gene]: # Itération sur chaque transcrit du gène
    fichier = f'../result/pre_fomonet_{gene}/{gene}_{transcrit}_mutations.tsv'
    with open(fichier, 'r') as file:
        for line in file:
            if line not in mutation_erronee:
                count_every_line += 1
        
if count_every_line + len(gene_dic[gene]) == len(orf):
    print('La quantité de données FOMOnet est correct')
else:
    print('La quantité de données FOMOnet est erroné')


# VÉRIFICATION DES SÉQUENCES PRÉDITES PAR FOMONET
#for transcrit in gene_dic[gene]: # Itération sur chaque transcrit
    #for key in orf: # Itération sur chaque clé du dictionnaire
        #if key == f'{gene}_{transcrit}': # Conserve uniquement les transcrits non-mutés
            #positions = orf[f'{gene}_{transcrit}'] # Retient les positions de début et de fin des transcrits
            #if positions != []: # Je ne veux pas les transcrits qui n'auraient pas de CDS
                #for tuple in positions:
                    #debut = tuple[0]
                    #fin = tuple[1]
                    #sequence = sequence_cDNA(transcrit)
                
                    # La séquence prédite par FOMOnet est donc:
                    #print(sequence[debut:fin])

# La section précédente permet de voir que la séquence prédite par FOMOnet commence par ATG et termine par TGA (ou autre codon STOP). Ainsi, si je
# veux uniquement conservé la séquence codante, je dois retirer le codon STOP qui correspond aux 3 derniers nucléotides de la chaine.

orf_sans_stop = {} # Création du dictionnaire vierge corrigé
for key in orf: # Itération sur chaque clé du dictionnaire orf
    orf_sans_stop[key]= [] # J'ajoute toutes les clés du dictionnaire orf dans orf_sans_stop et je met une liste vierge comme valeur
    for tuple in orf[key]:
        debut_cds = tuple[0] # Le début était déjà à la bonne valeur
        fin_cds = (tuple[1]-3) # Je fais -3 pour retirer le codon STOP
        tuple_corrige = (debut_cds, fin_cds) # Je rajoute les deux positions dans un tuple
        orf_sans_stop[key].append(tuple_corrige) # Dans la liste, j'ajoute le tuple avec les positions corrigées
        
# Vérification
# Je vérifie que la longueur du nouveau dictionnaire (corrigé) soit la même que la longueur de l'ancien dictionnaire

if len(orf_sans_stop)==len(orf):
    print('Correct!')
else:
    print('Error!')
          

La quantité de données FOMOnet est correct
Correct!


In [7]:
##### ÉTAPE 5: CRÉER DES DICTIONNAIRE RENFERMANT LES INFORMATIONS PERTINENTES DES MUTATIONS #####

# Chaque type de mutation a un dictionnaire, dont la clé est l'ID de la mutation et la valeur est une liste contenant la position génomique du début
# et de la fin de la mutation, les nucléotides qui sont ajoutés dans la séquence et/ou les nucléotides qui sont retirés de la séquence, ainsi que la
# nature du brin du gène. 
# Cette section vérifie également que le nombre d'élément contenu dans chaque dictionnaire correspond au nombre de chaque type de mutation.

# TRAITEMENT DES INVERSIONS
inversion = {} # Création du dictionnaire vierge
count_inversion = 0
fichier = f"../result/pre_fomonet_{gene}/{gene}_mutation_inversion.tsv"
try:
    with open(fichier, 'r') as file: # Ouverture du fichier qui contient toutes les mutations inversions du gène
        for line in file: # Itération sur chaque ligne du fichier
            count_inversion += 1
            colonne = line.split('\t') # Séparation avec les caractères tab
            mutation_id = colonne[8] # Stockage de l'identifiant de la mutation
            petit = int(colonne[15]) # Renferme la position génomique la plus petite
            grand = int(colonne[16]) # Renferme la position génomique la plus grande
            brand = colonne[17] # Renferme la nature du brin du gène
            delete = colonne[23] # Renferme la séquence de nucléotide enlevée
            ajoute = colonne[24] # Renferme la séquence de nucléotide ajoutée
            inversion[mutation_id] = [] # Dans le dictionnaire, j'ajoute la mutation_id comme clé et une liste vide comme valeur
            inversion[mutation_id].extend([petit, grand, ajoute, delete, brand]) # Ajoute des infos dans la liste
except FileNotFoundError: # Dans le cas, exemple, que le gène n'aurait pas de mutation inversion
    pass
if count_inversion == len(inversion):
    print('Correct for inversion!')
else:
    print('Error for inversion')


# TRAITEMENT DES SUBSTITUTIONS
substitution = {}
count_substitution = 0
fichier = f"../result/pre_fomonet_{gene}/{gene}_mutation_substitution.tsv"
try:
    with open(fichier, 'r') as file: # Ouverture du fichier qui contient toutes les mutations substitutions du gène
        for line in file: # Itération sur chaque ligne du fichier
            count_substitution += 1
            colonne = line.split('\t') # Séparation avec les caractères tab
            mutation_id = colonne[8] # Stockage de l'identifiant de la mutation
            petit = int(colonne[15]) # Renferme la position génomique la plus petite
            grand = int(colonne[16]) # Renferme la position génomique la plus grande
            brand = colonne[17] # Renferme la nature du brin du gène
            delete = colonne[23] # Renferme la séquence de nucléotide enlevée
            ajoute = colonne[24] # Renferme la séquence de nucléotide ajoutée
            substitution[mutation_id] = [] # Dans le dictionnaire, j'ajoute la mutation_id comme clé et une liste vide comme valeur
            substitution[mutation_id].extend([petit, grand, ajoute, delete, brand]) # Ajoute des infos dans la liste            
except FileNotFoundError:
    pass
if count_substitution == len(substitution):
    print('Correct for substitution!')
else:
    print('Error for substitution')


# TRAITEMENT DES INSERTIONS
insertion = {} # Création du dictionnaire vierge
count_insertion = 0
fichier = f"../result/pre_fomonet_{gene}/{gene}_mutation_insertion.tsv"
try:
    with open(fichier, 'r') as file: # Ouverture du fichier qui contient toutes les mutations insertions du gène
        for line in file: # Itération sur chaque ligne du fichier
            count_insertion += 1
            colonne = line.split('\t') # Séparation avec les caractères tab
            mutation_id = colonne[8] # Stockage de l'identifiant de la mutation
            petit = int(colonne[15]) # Renferme la position génomique la plus petite
            grand = int(colonne[16]) # Renferme la position génomique la plus grande
            brand = colonne[17] # Renferme la nature du brin du gène
            delete = colonne[23] # Renferme la séquence de nucléotide enlevée
            ajoute = colonne[24] # Renferme la séquence de nucléotide ajoutée
            insertion[mutation_id] = [] # Dans le dictionnaire, j'ajoute la mutation_id comme clé et une liste vide comme valeur
            insertion[mutation_id].extend([petit, grand, ajoute, delete, brand]) # Ajoute des infos dans la liste   
except FileNotFoundError:
    pass
if count_insertion == len(insertion):
    print('Correct for insertion!')
else:
    print('Error for insertion')


# TRAITEMENT DES DUPLICATIONS
duplication = {} # Création du dictionnaire vierge
count_duplication = 0
fichier = f"../result/pre_fomonet_{gene}/{gene}_mutation_duplication.tsv"
try:
    with open(fichier, 'r') as file: # Ouverture du fichier qui contient toutes les mutations insertions du gène
        for line in file: # Itération sur chaque ligne du fichier
            count_duplication += 1
            colonne = line.split('\t') # Séparation avec les caractères tab
            mutation_id = colonne[8] # Stockage de l'identifiant de la mutation
            petit = int(colonne[15]) # Renferme la position génomique la plus petite
            grand = int(colonne[16]) # Renferme la position génomique la plus grande
            brand = colonne[17] # Renferme la nature du brin du gène
            delete = colonne[23] # Renferme la séquence de nucléotide enlevée
            ajoute = colonne[24] # Renferme la séquence de nucléotide ajoutée
            duplication[mutation_id] = [] # Dans le dictionnaire, j'ajoute la mutation_id comme clé et une liste vide comme valeur
            duplication[mutation_id].extend([petit, grand, ajoute, delete, brand]) # Ajoute des infos dans la liste   
except FileNotFoundError:
    pass
if count_duplication == len(duplication):
    print('Correct for duplication!')
else:
    print('Error for duplication')

    
# TRAITEMENT DES DELETIONS
deletion = {} # Création du dictionnaire vierge
count_deletion = 0
fichier = f"../result/pre_fomonet_{gene}/{gene}_mutation_deletion.tsv"
try:
    with open(fichier, 'r') as file: # Ouverture du fichier qui contient toutes les mutations deletions du gène
        for line in file: # Itération sur chaque ligne du fichier
            count_deletion += 1
            colonne = line.split('\t') # Séparation avec les caractères tab
            mutation_id = colonne[8] # Stockage de l'identifiant de la mutation
            petit = int(colonne[15]) # Renferme la position génomique la plus petite
            grand = int(colonne[16]) # Renferme la position génomique la plus grande
            brand = colonne[17] # Renferme la nature du brin du gène
            delete = colonne[23] # Renferme la séquence de nucléotide enlevée
            ajoute = colonne[24] # Renferme la séquence de nucléotide ajoutée
            deletion[mutation_id] = [] # Dans le dictionnaire, j'ajoute la mutation_id comme clé et une liste vide comme valeur
            deletion[mutation_id].extend([petit, grand, ajoute, delete, brand]) # Ajoute des infos dans la liste   
except FileNotFoundError:
    pass
if count_deletion == len(deletion):
    print('Correct for deletion!')
else:
    print('Error for deletion')


# TRAITEMENT DES DELINS
del_ins = {} # Création du dictionnaire vierge
count_del_ins = 0
fichier = f"../result/pre_fomonet_{gene}/{gene}_mutation_del_ins.tsv"
try:
    with open(fichier, 'r') as file: # Ouverture du fichier qui contient toutes les mutations delins du gène
        for line in file: # Itération sur chaque ligne du fichier
            count_del_ins += 1
            colonne = line.split('\t') # Séparation avec les caractères tab
            mutation_id = colonne[8] # Stockage de l'identifiant de la mutation
            petit = int(colonne[15]) # Renferme la position génomique la plus petite
            grand = int(colonne[16]) # Renferme la position génomique la plus grande
            brand = colonne[17] # Renferme la nature du brin du gène
            delete = colonne[23] # Renferme la séquence de nucléotide enlevée
            ajoute = colonne[24] # Renferme la séquence de nucléotide ajoutée
            del_ins[mutation_id] = [] # Dans le dictionnaire, j'ajoute la mutation_id comme clé et une liste vide comme valeur
            del_ins[mutation_id].extend([petit, grand, ajoute, delete, brand]) # Ajoute des infos dans la liste   
except FileNotFoundError:
    pass
if count_del_ins == len(del_ins):
    print('Correct for delins!')
else:
    print('Error for delins')


# Je fais également un grand dictionnaire global qui contient toutes les informations pour toutes les mutations de tous les types.

all_mutations = substitution | inversion | insertion | duplication | deletion | del_ins
if (count_substitution + count_inversion + count_insertion + count_duplication + count_deletion + count_del_ins)== len(all_mutations):
    print('ALL CORRECT!')
else: 
    print('ERROR!')

Correct for inversion!
Correct for substitution!
Correct for insertion!
Correct for duplication!
Correct for deletion!
Correct for delins!
ALL CORRECT!


In [8]:
##### ÉTAPE 6: CORRECTION DES POSITIONS PRÉDITES PAR FOMONET ######

# Les positions données par FOMOnet sont par rapport à chaque transcrit muté. Si je veux pouvoir comparer les positions prédites après la mutation
# aux positions prédites (ou au CDS canonique) avant mutation, les positions après mutation doivent être rapporté selon le même référentiel que le
# transcrit non-muté. Je corrige donc toutes les positions des transcrits avec mutation.

orf_corrige = {} # Création du dictionnaire vierge qui contiendra les transcrits mutés avec les bonnes positions
for transcrit in gene_dic[gene]: # Itération sur chaque transcrit du gène
    for key in orf_sans_stop: # Itération sur chaque clé présent dans le dictionnaire
        if transcrit in key and key != f'{gene}_{transcrit}': # Je garde toutes les clés avec le transcrit à l'intérieur et AUSSI seulement, les transcrits qui sont mutés
            info = key.split('_') # Séparation avec les caractères _
            mutation_id = info[2] # Stockage de l'identifiant de la mutation


            # TRAITEMENT DES SUBSTITUTIONS
            # Les substitutions remplacent un nucléotide par un autre. Donc, le cadre de lecture n'est jamais décalé.
            # C'est-à-dire qu'aucune modification n'a besoin d'être apporté.
            
            if mutation_id in substitution:
                orf_corrige[key] = [] # Ajoute la clé au dictionnaire
                for tuple in orf_sans_stop[key]: # Itération sur chaque tuple présent pour la mutation
                    orf_corrige[key].append(tuple) # J'ajoute chaque tuple sans modification dans la liste


            # TRAITEMENT DES INVERSIONS
            # Les inversions correspondent à ce qu'un segment de la séquence soit inversée. Il n'y a pas de perte ni d'ajout de nucléotide.
            # Pas de décalage dans le cadre de lecture, donc aucune modification à apporter.
            
            elif mutation_id in inversion:
                orf_corrige[key] = [] # Ajoute la clé au dictionnaire
                for tuple in orf_sans_stop[key]: # Itération sur chaque tuple présent pour la mutation
                    orf_corrige[key].append(tuple) # J'ajoute chaque tuple sans modification dans la liste


            # TRAITEMENT DES INSERTIONS
            # L'insertion de nucléotide provoque un décalage dans le cadre de lecture
            
            elif mutation_id in insertion:
                ajoute = int(len(insertion[mutation_id][2])) # Le nombre de nucléotide ajouté
                petit = int(insertion[mutation_id][0]) # Petite position génomique de la mutation
                grand = int(insertion[mutation_id][1]) # Grande position génomique de la mutation
                petit_chrono = (get_position_chronologique(petit, transcrit))-1 # Conversion génomique -> chronologique commençant par 0
                grand_chrono = (get_position_chronologique(grand, transcrit))-1 # Commence par 0
                position_chrono = [petit_chrono, grand_chrono] # Ajoute les deux positions chronologique dans une liste
                smallest = min(position_chrono) # Sort la plus petite position chronologique = début de la mutation
                biggest = max(position_chrono) # Sort la plus grande position chronologique = fin de la mutation
                orf_corrige[key] = [] # Ajoute la clé comme clé dans le dictionnaire avec une liste vierge
                for tuple in orf_sans_stop[key]: # Itération sur chaque tuple pour la mutation
                    debut_cds = tuple[0] # Position début du cds prédit
                    fin_cds = (int(tuple[1])-1) # Position fin du cds prédit, -1 pour vraiment aller chercher la vraie position et pas la position+1
                    
                    # CONVERSION DE CES POSITIONS

                    # Si la position est avant l'insertion = PAS DE CHANGEMENT
                    if debut_cds <= smallest:
                        debut_cds_corrige = debut_cds
                    # Si la position se trouve dans la mutation = ON NE PEUT PAS Y ASSOCIÉ UNE POSITION DANS LE TRANSCRIT RÉFÉRENTIEL
                    elif smallest < debut_cds <= (smallest + ajoute):
                        reste = debut_cds-smallest
                        debut_cds_corrige = str(smallest) + '+' + str(reste)
                    # Si la position est après l'insertion
                    else:
                        if debut_cds > (smallest + ajoute):
                            debut_cds_corrige = (debut_cds-ajoute)

                    # Si la position est avant l'insertion = PAS DE CHANGEMENT
                    if fin_cds <= smallest:
                        fin_cds_corrige = fin_cds+1 # Je rajoute 1, car la position de fin n'est pas incluse sur Python
                    # Si la position se trouve dans la mutation = ON NE PEUT PAS Y ASSOCIÉ UNE POSITION DANS LE TRANSCRIT RÉFÉRENTIEL
                    elif smallest < fin_cds <= (smallest + ajoute):
                        reste = fin_cds-smallest+1
                        fin_cds_corrige = str(smallest) + '+' + str(reste)
                    # Si la position est après l'insertion
                    else:
                        if fin_cds > (smallest + ajoute):
                            fin_cds_corrige = (fin_cds-ajoute+1)
                    tuple_corrige = (debut_cds_corrige, fin_cds_corrige) # Ajoute les positions corrigées dans un tuple
                    orf_corrige[key].append(tuple_corrige) # Ajoute le tuple dans la liste

            # TRAITEMENT DES DUPLICATIONS
            # Les duplications se traitent exactement de la même manière que les insertions: elles ajoutent des nucléotides dans les séquences
            
            elif mutation_id in duplication:
                ajoute = int(len(duplication[mutation_id][2])) # Stockage du nombre de nucléotide dupliquée
                petit = int(duplication[mutation_id][0]) # Stockage de la plus petite position génomique
                grand = int(duplication[mutation_id][1]) # Stockage de la plus grande position génomique
                petit_chrono = (get_position_chronologique(petit, transcrit))-1 # Conversion génomique -> chronologique en commençant par 0
                grand_chrono = (get_position_chronologique(grand, transcrit))-1 # Commence par 0
                position_chrono = [petit_chrono, grand_chrono] # Ajoute les positions chronologique dans une liste
                smallest = min(position_chrono) # Sort la plus petite position chronologique = début de la mutation
                biggest = max(position_chrono) # Sort la plus grande position chronologique = fin de la mutation
                orf_corrige[key] = [] # Ajoute la clé au dictionnaire et une liste vierge comme valeur
                for tuple in orf_sans_stop[key]: # Itération sur chaque tuple de la mutation
                    debut_cds = tuple[0] # Position début cds prédit
                    fin_cds = (tuple[1]-1) # Position fin cds prédit
                    
                    # CONVERSION DE CES POSITIONS

                    # Si la position est avant la duplication = PAS DE CHANGEMENT
                    if debut_cds <= smallest:
                        debut_cds_corrige = debut_cds
                    # Si la position se trouve dans la mutation = ON NE PEUT PAS Y ASSOCIÉ UNE POSITION DANS LE TRANSCRIT RÉFÉRENTIEL
                    elif smallest < debut_cds <= (smallest + ajoute):
                        reste = debut_cds-smallest
                        debut_cds_corrige = str(smallest) + '+' + str(reste)
                    else:
                        # Si la position est après la duplication
                        if debut_cds > (smallest + ajoute):
                            debut_cds_corrige = (debut_cds-ajoute)
                    # Si la position est avant la duplication = PAS DE CHANGEMENT
                    if fin_cds <= smallest:
                        fin_cds_corrige = fin_cds+1
                    # Si la position se trouve dans la mutation = ON NE PEUT PAS Y ASSOCIÉ UNE POSITION DANS LE TRANSCRIT RÉFÉRENTIEL
                    elif smallest < fin_cds <= (smallest + ajoute):
                        reste = fin_cds-smallest+1
                        fin_cds_corrige = str(smallest) + '+' + str(reste)
                    else:
                        # Si la position est après la duplication
                        if fin_cds > (smallest + ajoute):
                            fin_cds_corrige = (fin_cds-ajoute+1)
                    tuple_corrige = (debut_cds_corrige, fin_cds_corrige) # Ajoute les positions corrigées dans un tuple
                    orf_corrige[key].append(tuple_corrige) # Ajoute le tuple dans la liste


            # TRAITEMENT DES DELETIONS
            # Les deletions supprime des nucléotides. Elles décalent le cadre de lecture.
            
            elif mutation_id in deletion:
                delete = int(len(deletion[mutation_id][3])) # Stockage du nombre de nucléotide enlevé
                petit = int(deletion[mutation_id][0]) # Stockage de la plus petite position génomique
                grand = int(deletion[mutation_id][1]) # Stockage de la plus grande position génomique
                petit_chrono = (get_position_chronologique(petit, transcrit))-1 # Conversion génomique -> chronologique commençant par 0
                grand_chrono = (get_position_chronologique(grand, transcrit))-1 # Commence par 0
                position_chrono = [petit_chrono, grand_chrono] # Ajoute dans les deux positions chronologique dans une liste
                smallest = min(position_chrono) # Sort la position chronologique la plus petite = PREMIER NUCLÉOTIDE QUI EST RETIRÉ
                biggest = max(position_chrono) # Sort la position chronologique la plus grande = DERNIER NUCLÉOTIDE QUI EST RETIRÉ
                orf_corrige[key] = [] # Ajoute la clé au dictionnaire avec une liste vierge comme valeur
                for tuple in orf_sans_stop[key]: # Itération sur chaque tuple de la mutation
                    debut_cds = tuple[0] # Position début cds prédit
                    fin_cds = (tuple[1]-1) # Position fin cds prédit

                    # Si la position est avant la deletion = PAS DE CHANGEMENT
                    if debut_cds <= smallest-1: 
                        debut_cds_corrige = debut_cds
                    # Si la position est après la deletion
                    else:
                        if debut_cds > smallest-1:
                            debut_cds_corrige = (debut_cds+delete)
                    # Si la position est avant la deletion = PAS DE CHANGEMENT
                    if fin_cds <= smallest-1:
                        fin_cds_corrige = fin_cds+1
                    # Si la position est après la deletion
                    else:
                        if fin_cds > smallest-1:
                            fin_cds_corrige = (fin_cds+delete+1)
                    tuple_corrige = (debut_cds_corrige, fin_cds_corrige) # Ajoute les positions corrigées au tuple
                    orf_corrige[key].append(tuple_corrige) # Ajoute le tuple dans la liste

            # TRAITEMENT DES DELINS
            else:
                if mutation_id in del_ins:
                    ajoute = int(len(del_ins[mutation_id][2])) # Stockage du nombre de nucléotide ajouté
                    delete = int(len(del_ins[mutation_id][3])) # Stockage du nombre de nucléotide enlevé
                    petit = int(del_ins[mutation_id][0]) # Stockage de la plus petite position génomique
                    grand = int(del_ins[mutation_id][1]) # Stockage de la plus grande position génomique
                    petit_chrono = (get_position_chronologique(petit, transcrit))-1 # Conversion génomique -> chronologique commençant par 0
                    grand_chrono = (get_position_chronologique(grand, transcrit))-1 # Commence par 0
                    position_chrono = [petit_chrono, grand_chrono] # Ajoute les positions chronologiques dans une liste
                    smallest = min(position_chrono) # Sort la plus petite position chronologique = PREMIER NUCLÉOTIDE RETIRÉ
                    biggest = max(position_chrono) # Sort la plus grande position chronologique = DERNIER NUCLÉOTIDE RETIRÉ
                    orf_corrige[key] = [] # Ajoute la clé au dictionnaire avec une liste vierge
                    for tuple in orf_sans_stop[key]: # Itération sur chaque tuple de la mutation
                        debut_cds = tuple[0] # Position début cds prédit
                        fin_cds = (tuple[1]-1) # Position fin cds prédit

                        # PAS DE DÉCALAGE DU CADRE DE LECTURE
                        if ajoute == delete:
                            debut_cds_corrige = debut_cds
                            fin_cds_corrige = fin_cds+1
                    
                        else:
                            # Si la position est avant la mutation = PAS DE CHANGEMENT 
                            if debut_cds <= smallest-1:
                                debut_cds_corrige = debut_cds
                            # Si la position tombe dans la mutation
                            elif smallest-1 < debut_cds <= (smallest-1+ajoute):
                                reste = debut_cds-smallest-1
                                smallest_corr = smallest-1
                                debut_cds_corrige = str(smallest_corr) + '+' + str(reste)
                            else:
                                # Si la position est après la mutation
                                if debut_cds > (smallest-1+ajoute):
                                    debut_cds_corrige = (debut_cds + delete - ajoute)

                            # Si la position est avant la mutation = PAS DE CHANGEMENT 
                            if fin_cds <= smallest-1:
                                fin_cds_corrige = fin_cds+1
                            # Si la position tombe dans la mutation
                            elif smallest-1 < fin_cds <= (smallest-1+ajoute):
                                reste = fin_cds-smallest-1+1
                                smallest_corr = smallest-1
                                fin_cds_corrige = (smallest_corr) + '+' + str(reste)
                            else:
                                # Si la position est après la mutation
                                if fin_cds > (smallest-1+ajoute):
                                    fin_cds_corrige = (fin_cds + delete-ajoute+1)
                        tuple_corrige = (debut_cds_corrige, fin_cds_corrige) # Ajoute les positions dans un tuple
                        orf_corrige[key].append(tuple_corrige) # Ajoute le tuple dans la liste
                        
# Vérification
if (len(orf_sans_stop)-len(gene_dic[gene])) == len(orf_corrige):
    print('Correct!')
else:
    print('Error!')


Correct!


In [9]:
##### ÉTAPE 7: PRÉPARATION DES DONNÉES QUI SE RETROUVERONT DANS LE FICHIER D'ANALYSE DU GÈNE #####

# Je crée des fonctions qui permetteront de sortir les données qui se retrouveront dans le fichier d'analyse des données prédites par FOMOnet. Ces
# fonctions seront utilisés lors de la création du fichier.

# COLONNE 2: Donne le CDS canonique du transcrit sans mutation
def get_cds_canonique(transcrit):
    try:
        premier_exon = list(cds_dic[gene][transcrit])[0] # Stockage du premier exon du transcrit
        dernier_exon = list(cds_dic[gene][transcrit])[-1] #Stockage du dernier exon du transcrit
        if brand == '-':
            debut_cds = int(cds_dic[gene][transcrit][premier_exon][1]) 
            fin_cds = int(cds_dic[gene][transcrit][dernier_exon][0])
            debut = (get_position_chronologique(debut_cds, transcrit))-1 # Conversion position chronologique + j'ajuste la position pour qu'elle commence à 0 et non à 1.
            fin = get_position_chronologique(fin_cds, transcrit) # Conversion position chronologique: je ne fais pas -1, car la position est excluse de la séquence
            position_cds = [(debut, fin)] # J'ajoute les positions de début et de fin à un tuple

        elif brand == '+':
            debut_cds = int(cds_dic[gene][transcrit][premier_exon][0])
            fin_cds = int(cds_dic[gene][transcrit][dernier_exon][1])
            debut = (get_position_chronologique(debut_cds, transcrit))-1 # Conversion position chronologique + j'ajuste la position pour qu'elle commence à 0 et non à 1.
            fin = get_position_chronologique(fin_cds, transcrit) # Conversion position chronologique: je ne fais pas -1, car la position est excluse de la séquence
            position_cds = [(debut, fin)] # J'ajoute les positions de début et de fin à un tupple

    except IndexError: # Parfois, il n'y a pas de CDS annoté pour le transcrit, je laisse une liste vide
        position_cds = [] 
    return position_cds


# COLONNE 3: Donne le ou les CDS prédits par FOMOnet
def get_fomonet_wt(transcrit):
    position_fomonet = orf_sans_stop[f'{gene}_{transcrit}']
    return position_fomonet


# COLONNE 5: Donne le type de mutation qui a été effectué dans le transcrit
def get_mutation_type(key):
    info = key.split('_') # Séparation avec les caractères _
    mutation_id = info[2] # Stock l'ID de la mutation

    # Détermine l'ID de la mutation correspond à quel type de mutation
    if mutation_id in substitution:
        mutation_type = 'substitution'
    elif mutation_id in insertion:
        mutation_type = 'insertion'
    elif mutation_id in deletion:
        mutation_type = 'deletion'
    elif mutation_id in del_ins:
        mutation_type = 'del_ins'
    elif mutation_id in duplication:
        mutation_type = 'duplication'
    else:
        mutation_type = 'inversion'
    return mutation_type


# COLONNE 6: Donne la catégorie de la mutation (selon COSMICS)
def get_mutation_cat(key):
    info = key.split('_')
    mutation_id = info[2]
    fichier = f"../result/pre_fomonet_{gene}/{gene}_toutes_les_mutations_exons.tsv"
    with open(fichier, 'r') as file:
        for line in file:
            colonne = line.split('\t')
            if colonne[8]== mutation_id:
                mutation_cat = colonne[11]
                return mutation_cat
            


# COLONNE 9: Retourne la position de départ de la mutation
def get_position_debut(key):
    info = key.split('_')
    transcrit = info[1]
    mutation_id = info[2]
    petite_chromo = all_mutations[mutation_id][0]
    grande_chromo = all_mutations[mutation_id][1]
    chrono_1 = (get_position_chronologique(petite_chromo, transcrit))-1 # -1 Pour commencer à 0
    chrono_2 = (get_position_chronologique(grande_chromo, transcrit))-1 # -1 Pour commencer à 0
    positions = [chrono_1, chrono_2]
    debut = min(positions)
    return debut


# COLONNE 10: Retourne la position de fin de la mutation
def get_position_fin(key):
    info = key.split('_')
    transcrit = info[1]
    mutation_id = info[2]
    petite_chromo = all_mutations[mutation_id][0]
    grande_chromo = all_mutations[mutation_id][1]
    chrono_1 = (get_position_chronologique(petite_chromo, transcrit))-1 # -1 Pour commencer à 0
    chrono_2 = (get_position_chronologique(grande_chromo, transcrit))-1 # -1 Pour commencer à 0
    positions = [chrono_1, chrono_2]
    fin = max(positions)
    return fin


# COLONNE 11: Détermine la séquence de nucléotide qui est ajouté
def get_whats_add(key):
    info = key.split('_')
    mutation_id = info[2]
    # Détermine la nature du brin
    brand =  all_mutations[mutation_id][4]
    
    if brand == '-':
        ajoute_genome = all_mutations[mutation_id][2]
        chaine_inverse = ajoute_genome[::-1] # Inversion (miroir) de la chaine
        # Obtention de la séquence complémentaire 
        adn1 = chaine_inverse.replace('A', 't') 
        adn2 = adn1.replace('T', 'a')
        adn3 = adn2.replace('C', 'g')
        adn4 = adn3.replace('G', 'c')
        ajoute = adn4.upper() # Change les caractères en majuscule
    
    elif brand == '+':
        ajoute = all_mutations[mutation_id][2] # La bonne séquence est directement donnée
        
    return ajoute


# COLONNE 12: Détermine la séquence de nucléotide qui est deleté
def get_whats_delete(key):
    info = key.split('_')
    mutation_id = info[2]
    # Détermine la nature du brin
    brand =  all_mutations[mutation_id][4]
    
    if brand == '-':
        delete_genome = all_mutations[mutation_id][3]
        chaine_inverse = delete_genome[::-1] # Inversion (miroir) de la chaine
        # Obtention de la séquence complémentaire 
        adn1 = chaine_inverse.replace('A', 't') 
        adn2 = adn1.replace('T', 'a')
        adn3 = adn2.replace('C', 'g')
        adn4 = adn3.replace('G', 'c')
        delete = adn4.upper() # Change les caractères en majuscule
    
    elif brand == '+':
        delete = all_mutations[mutation_id][3] # La bonne séquence est directement donnée
        
    return delete


# COLONNE 13: Détermine si la mutation tombe dans la séquence du CDS canonique
def is_mutation_in_cds_canonique(key):
    info = key.split('_')
    transcrit = info[1]
    mutation_id = info[2]
    debut = get_position_debut(key)
    fin = get_position_fin(key)
    cds_canonique = get_cds_canonique(transcrit)
    
    # Il y a 0 ou 1 CDS canonique
    if cds_canonique != []:
        for tupple in cds_canonique:
            debut_cds = tupple[0]
            fin_cds = tupple[1]-1 # -1 Parce que la position donnée est excluse. Je veux la vrai position de fin
            if mutation_id in substitution: # La position de début et de fin doit se retrouver dans le CDS
                if (debut_cds<=debut and fin_cds>=debut) and (debut_cds<=fin and fin_cds>=fin):
                    in_or_out = 'yes'
                else:
                    in_or_out = 'no'
                    
            elif mutation_id in inversion or mutation_id in deletion or mutation_id in del_ins: # Une des deux positions seulement peut se retrouver dans la séquence
                if (debut_cds<=debut and fin_cds>=debut) or (debut_cds<=fin and fin_cds>=fin):
                    in_or_out = 'yes'
                else:
                    in_or_out = 'no'
                    
            elif mutation_id in duplication or mutation_id in insertion:
                if fin_cds == debut:
                    in_or_out = 'no'
                elif debut_cds == fin:
                    in_or_out = 'no'
                else:
                    if (debut_cds<=debut and fin_cds>debut) or (debut_cds<fin and fin_cds>=fin):
                        in_or_out = 'yes'
                    else:
                        in_or_out = 'no'
                
    elif cds_canonique == []:
        in_or_out = 'no'
    return in_or_out


# COLONNE 14: Détermine si la mutation tombe dans le CDS prédit par FOMOnet (si CDS_canonique == fomonet_wt, la colonne contient False)
def is_mutation_in_fomonet_wt(key):
    info = key.split('_')
    transcrit = info[1]
    mutation_id = info[2]
    debut = get_position_debut(key)
    fin = get_position_fin(key)
    cds_canonique = get_cds_canonique(transcrit)
    fomonet_wt = get_fomonet_wt(transcrit)
    
    if cds_canonique == fomonet_wt:
        in_or_out = False
    else:
        in_or_out = ''
        # Conserve uniquement les cas où CDS canonique != FOMOnet_wt
        # Attention! FOMOnet_wt peut avoir plusieurs CDS de prédit

        if fomonet_wt != []:
            for tuple in fomonet_wt:
                debut_cds = tuple[0]
                fin_cds = tuple[1]-1 # -1 Parce que la position donnée est excluse. Je veux la vrai position de fin
                if mutation_id in substitution: # La position de début et de fin doit se retrouver dans le CDS
                    if (debut_cds<=debut and fin_cds>=debut) and (debut_cds<=fin and fin_cds>=fin):
                        in_or_out = 'yes'
                    else:
                        in_or_out = 'no'
                    
                elif mutation_id in inversion or mutation_id in deletion or mutation_id in del_ins: # Une des deux positions seulement peut se retrouver dans la séquence
                    if (debut_cds<=debut and fin_cds>=debut) or (debut_cds<=fin and fin_cds>=fin):
                        in_or_out = 'yes'
                    else:
                        in_or_out = 'no'
                    
                elif mutation_id in duplication or mutation_id in insertion:
                    if fin_cds == debut:
                        in_or_out = 'no'
                    elif debut_cds == fin:
                        in_or_out = 'no'
                    else:
                        if (debut_cds<=debut and fin_cds>debut) or (debut_cds<fin and fin_cds>=fin):
                            in_or_out = 'yes'
                        else:
                            in_or_out = 'no'            
        
        elif fomonet_wt == []:
            in_or_out += 'no'
    return in_or_out



# COLONNE 15: Déterminer le codon START du CDS canonique
def get_codon_start_canonique(transcrit):
    sequence = sequence_cDNA(transcrit)
    position_cds = get_cds_canonique(transcrit) # Sort le CDS canonique: un tuple ou une liste vide
    if position_cds != []:
        for tupple in position_cds:
            debut = tupple[0]
            fin = tupple[0]+3
            codon_start = sequence[debut:fin]
    if position_cds == []:
        codon_start = None
    return codon_start


# COLONNE 16: Déterminer le codon START du CDS canonique
def get_codon_start_fomonet_wt(transcrit):
    sequence = sequence_cDNA(transcrit)
    position_fomonet = get_fomonet_wt(transcrit) # Sort la prédiction FOMOnet (liste)
    if position_fomonet != []:
        nombre_iteration = 0
        codon_start = ''
        for tuple in position_fomonet:
            nombre_iteration += 1
            debut = tuple[0]
            fin = tuple[0]+3
            codon_start += sequence[debut:fin]
            
            if nombre_iteration > 1:
                codon_start += ', '

    if position_fomonet == []:
        codon_start = None
    return codon_start


# COLONNE 17: Déterminer les codons START des CDS prédit par FOMOnet après mutation
def get_codon_start_mute(key):
    info = key.split('_')
    transcrit = info[1]
    mutation_id = info[2]
    sequence = sequence_mute(transcrit, mutation_id)
    position_mute = orf_sans_stop[key] # Sort la prédiction FOMOnet (liste)
    if position_mute != []:
        nombre_iteration = 0
        codon_start = ''
        for tuple in position_mute:
            nombre_iteration += 1
            debut = tuple[0]
            fin = tuple[0]+3
            codon_start += sequence[debut:fin]
            
            if nombre_iteration > 1:
                codon_start += ', '

    if position_mute == []:
        codon_start = None
    return codon_start

    


In [10]:
# COLONNE 7: Donne la comparaison entre le CDS canonique et la prédiction FOMOnet du transcrit muté
def get_comparaison_canonical_vs_fomonet_mute(key):
    info = key.split('_') # Séparation dans la clé
    transcrit = info[1] # Stockage du nom du transcrit ENST
    cds_canonique = get_cds_canonique(transcrit) # Sort le CDS canonique du transcrit
    prediction_mute = orf_corrige[key] # Sort la prédiction fomonet du transcrit muté

    # Le transcrit peut avoir 0 ou 1 CDS canonique.

    # Le transcrit à 1 CDS canonique
    if len(cds_canonique) == 1:

        # Après la mutation, FOMOnet ne prédit plus de CDS
        if len(prediction_mute) == 0:
            canonique_vs_fomonet_mute = 'loss of CDS' # Il y a alors, perte du CDS canonique

        # Après la mutation, FOMOnet prédit 1 CDS
        elif len(prediction_mute) == len(cds_canonique):

            # Il faut comparer le CDS prédit au CDS canonique
            for canonical in cds_canonique: # Itération sur le tuple du CDS canonique
                for predic in prediction_mute: # Itération sur le tuple de la prédiction FOMOnet

                    if isinstance(predic[0], int): # La position de départ est un int (c'est-à-dire, pas dans la mutation)
                        if isinstance(predic[1], int): # La position de fin est un int (c'est-à-dire, pas dans la mutation)

                            # Je peux avoir confiance en la position de départ et en la position de fin

                            # Le CDS canonique correspond au CDS prédit après mutation
                            if canonical[0]==predic[0] and canonical[1]==predic[1]:
                                canonique_vs_fomonet_mute = 'normal'

                            # Le CDS prédit commence au même endroit que le CDS canonique
                            elif canonical[0]==predic[0] and canonical[1]!=predic[1]:

                                # Détermine s'il s'agit d'une élongation ou d'une troncation
                                if predic[1]>canonical[1]:
                                    canonique_vs_fomonet_mute = 'elongation'
                                elif predic[1]<canonical[1]:
                                    canonique_vs_fomonet_mute = 'troncation'
                            
                            # Le CDS prédit termine au même endroit que le CDS canonique
                            elif canonical[1]==predic[1] and canonical[0]!=predic[0]:

                                # Détermine s'il s'agit d'une élongation ou d'une troncation
                                if predic[0]>canonical[0]:
                                    canonique_vs_fomonet_mute = 'troncation'
                                elif predic[0]<canonical[0]:
                                    canonique_vs_fomonet_mute = 'elongation'

                            else:
                                # Aucune position en commun avec le CDS canonique
                                if canonical[0]!=predic[0] and canonical[1]!=predic[1]:
                                    if (predic[0]-canonical[0]) % 3 == 0 and (predic[1]-canonical[1]) % 3 == 0:
                                        canonique_vs_fomonet_mute = 'inframe'
                                    else:
                                        # Je met 'novel CDS' même lorsque selon le début ou la fin c'est un inframe. (Cas où FRAMESHIFT à l'intérieur de la séquence)
                                        canonique_vs_fomonet_mute = 'novel CDS'


                        elif isinstance(predic[1], str): # La position de fin tombe dans la mutation
                            position = predic[1].split('+')
                            predic_fin = int(position[0]) # Stockage de la dernière position faisant partie du transcrit de référence

                            # J'ai seulement confiance en la position de départ, car la position de fin n'est pas vraiment la bonne
                            # Les CDS ont la même position de départ
                            if predic[0]==canonical[0]: 

                                # Je peux déterminer s'il s'agit d'une troncation ou d'une élongation
                                if predic_fin>=canonical[1]:
                                    canonique_vs_fomonet_mute = 'elongation'
                                elif predic_fin<canonical[1]:
                                    canonique_vs_fomonet_mute = 'troncation'

                            # Si le départ n'est pas le même
                            else:
                                # Je dois trouver si c'est inframe ou frameshift -> J'ai seulement confiance en la position de départ
                                if (predic[0]-canonical[0]) % 3 == 0:
                                    canonique_vs_fomonet_mute = 'inframe'
                                else: 
                                    canonique_vs_fomonet_mute = 'novel CDS'

                    elif isinstance(predic[0], str): # Si la position de départ tombe dans la mutation
                        if isinstance(predic[1], int): # La position de fin ne tombe pas dans la mutation
                            position = predic[0].split('+')
                            predic_debut = int(position[0])

                            # Je ne peux pas avoir confiance en la position de départ
                            # Les CDS termine à la même position
                            if predic[1]==canonical[1]:

                                # Je peux déterminer s'il s'agit d'une troncation ou d'une élongation
                                if predic_debut>=canonical[0]:
                                    canonique_vs_fomonet_mute = 'troncation'
                                elif predic_debut<canonical[0]:
                                    canonique_vs_fomonet_mute = 'elongation'

                            # Les CDS n'ont pas la même fin
                            else:
                                # Je dois trouver si c'est inframe ou frameshift -> J'ai seulement confiance en la position de fin
                                if (predic[1]-canonical[1]) % 3 == 0:
                                    canonique_vs_fomonet_mute = 'inframe'
                                else: 
                                    canonique_vs_fomonet_mute = 'novel CDS'


                        # Je ne pense pas que ce cas soit possible, ou sinon très rare: je le traite quand même au cas où
                        elif isinstance(predic[1], str): # Dans le cas où les deux positions tombent dans la mutation
                            # Le CDS prédit est entièrement situé dans, par exemple une longue insertion. Il s'agit d'une séquence complètement nouvelle
                            canonique_vs_fomonet_mute = 'novel CDS intra-mutation'

        # Après mutation, FOMOnet prédit plusieurs CDS
        elif len(prediction_mute) > len(cds_canonique):
            canonique_vs_fomonet_mute = 'Gain de CDS : ' # FOMOnet prédit un gain de CDS

        
            for canonical in cds_canonique: # Itération sur le tuple du CDS canonique
                nombre_iteration = 0 # Indique le nombre d'itération autour de prediction_mute. Chaque fois qu'on passe à un prochain, il va augmenter de 1
                for predic in prediction_mute: # Itération sur les tuples contenus dans prediction_mute
                    nombre_iteration = nombre_iteration + 1 # Nouvelle itération = +1
                        
                    if nombre_iteration > 1: # Quand le nombre d'itération est supérieur à 1. 
                        canonique_vs_fomonet_mute += ' + ' # Ajoute un + entre les éléments

                    
                    if isinstance(predic[0], int): # La position de départ est un int (c'est-à-dire, pas dans la mutation)
                        if isinstance(predic[1], int): # La position de fin est un int (c'est-à-dire, pas dans la mutation)

                            # Je peux avoir confiance en la position de départ et en la position de fin

                            # Le CDS canonique correspond au CDS prédit après mutation
                            if canonical[0]==predic[0] and canonical[1]==predic[1]:
                                canonique_vs_fomonet_mute += 'normal'

                            # Le CDS prédit commence au même endroit que le CDS canonique
                            elif canonical[0]==predic[0] and canonical[1]!=predic[1]:

                                # Détermine s'il s'agit d'une élongation ou d'une troncation
                                if predic[1]>canonical[1]:
                                    canonique_vs_fomonet_mute += 'elongation'
                                elif predic[1]<canonical[1]:
                                    canonique_vs_fomonet_mute += 'troncation'
                            
                            # Le CDS prédit termine au même endroit que le CDS canonique
                            elif canonical[1]==predic[1] and canonical[0]!=predic[0]:

                                # Détermine s'il s'agit d'une élongation ou d'une troncation
                                if predic[0]>canonical[0]:
                                    canonique_vs_fomonet_mute += 'troncation'
                                elif predic[0]<canonical[0]:
                                    canonique_vs_fomonet_mute += 'elongation'

                            else:
                                # Aucune position en commun avec le CDS canonique
                                if canonical[0]!=predic[0] and canonical[1]!=predic[1]:
                                    if (predic[0]-canonical[0]) % 3 == 0 and (predic[1]-canonical[1]) % 3 == 0:
                                        canonique_vs_fomonet_mute += 'inframe'
                                    else:
                                        canonique_vs_fomonet_mute += 'novel CDS'


                        elif isinstance(predic[1], str): # La position de fin tombe dans la mutation
                            position = predic[1].split('+')
                            predic_fin = int(position[0]) # Stockage de la dernière position faisant partie du transcrit de référence

                            # J'ai seulement confiance en la position de départ, car la position de fin n'est pas vraiment la bonne
                            # Les CDS ont la même position de départ
                            if predic[0]==canonical[0]: 

                                # Je peux déterminer s'il s'agit d'une troncation ou d'une élongation
                                if predic_fin>=canonical[1]:
                                    canonique_vs_fomonet_mute += 'elongation'
                                elif predic_fin<canonical[1]:
                                    canonique_vs_fomonet_mute += 'troncation'

                            # Si le départ n'est pas le même
                            else:
                                # Je dois trouver si c'est inframe ou frameshift -> J'ai seulement confiance en la position de départ
                                if (predic[0]-canonical[0]) % 3 == 0:
                                    canonique_vs_fomonet_mute += 'inframe'
                                else: 
                                    canonique_vs_fomonet_mute += 'novel CDS'

                    elif isinstance(predic[0], str): # Si la position de départ tombe dans la mutation
                        if isinstance(predic[1], int): # La position de fin ne tombe pas dans la mutation
                            position = predic[0].split('+')
                            predic_debut = int(position[0])

                            # Je ne peux pas avoir confiance en la position de départ
                            # Les CDS termine à la même position
                            if predic[1]==canonical[1]:

                                # Je peux déterminer s'il s'agit d'une troncation ou d'une élongation
                                if predic_debut>=canonical[0]:
                                    canonique_vs_fomonet_mute += 'troncation'
                                elif predic_debut<canonical[0]:
                                    canonique_vs_fomonet_mute += 'elongation'

                            # Les CDS n'ont pas la même fin
                            else:
                                # Je dois trouver si c'est inframe ou frameshift -> J'ai seulement confiance en la position de fin
                                if (predic[1]-canonical[1]) % 3 == 0:
                                    canonique_vs_fomonet_mute += 'inframe'
                                else: 
                                    canonique_vs_fomonet_mute += 'novel CDS'

                        elif isinstance(predic[1], str): # Dans le cas où les deux positions tombent dans la mutation
                            # Le CDS prédit est entièrement situé dans, par exemple une longue insertion. Il s'agit d'une séquence complètement nouvelle
                            canonique_vs_fomonet_mute += 'novel CDS intra-mutation'

    # Finalement, si le transcrit n'a pas de CDS canonique
    elif len(cds_canonique)==0:

        # FOMOnet ne prédit aucun CDS
        if len(cds_canonique)==len(prediction_mute): 
            canonique_vs_fomonet_mute = 'normal'

        # FOMOnet prédit un CDS: il s'agit d'un gain de CDS    
        elif len(cds_canonique)<len(prediction_mute):
            x = len(prediction_mute) # Stock le nombre de CDS prédit par FOMOnet
            canonique_vs_fomonet_mute = f"Gain de CDS ({x})"
    return canonique_vs_fomonet_mute



# COLONNE 8: Détermine si la mutation tombe dans le CDS prédit par FOMOnet
def get_comparaison_fomonet_wt_vs_fomonet_mute(key):
    info = key.split('_') # Séparation dans la clé
    gene = info[0] # Stockage du nom du gène ENSG
    transcrit = info[1] # Stockage du nom du transcrit ENST
    cds_canonique = get_cds_canonique(transcrit) # Sort le CDS canonique du transcrit
    fomonet_wt = orf_sans_stop[f'{gene}_{transcrit}'] # Sort la prédiction FOMOnet du transcrit sans mutation
    prediction_mute = orf_corrige[key] # Sort la prédiction FOMOnet du transcrit avec mutation

    # Si le CDS canonique correspond exactement à la prédiction de FOMOnet, je n'ai pas besoin d'effectuer la comparaison
    if fomonet_wt == cds_canonique:
        fomonet_wt_vs_fomonet_mute = False
    # Si le CDS canonique ne correspond pas à la prédiction de FOMOnet, je vais faire la comparaison wt/mute
    else:
        fomonet_wt_vs_fomonet_mute = True # Éventuellement, tous les True doivent disparaitre

        # FOMOnet prédit 1 CDS pour le wild-type
        if len(fomonet_wt) == 1:

            # Après la mutation, FOMOnet ne prédit plus de CDS
            if len(prediction_mute) == 0:
                fomonet_wt_vs_fomonet_mute = 'loss of CDS' # Il y a perte du CDS wt prédit
            
            # Après la mutation, FOMOnet prédit 1 CDS
            if len(prediction_mute)== len(fomonet_wt):

                # Il faut comparer s'il s'agit des mêmes CDS prédit
                for wildtype in fomonet_wt: # Itération sur le tuple du CDS wild-type
                    for predic in prediction_mute: # Itération sur le tuple de la prédiction FOMOnet

                        if isinstance(predic[0], int): # La position de départ est un int (c'est-à-dire, pas dans la mutation)
                            if isinstance(predic[1], int): # La position de fin est un int (c'est-à-dire, pas dans la mutation)

                            # Je peux avoir confiance en la position de départ et en la position de fin

                                # Le CDS wild-type correspond au CDS prédit après mutation
                                if wildtype[0]==predic[0] and wildtype[1]==predic[1]:
                                    fomonet_wt_vs_fomonet_mute = 'normal'

                                # Le CDS prédit commence au même endroit que le CDS wild-type
                                elif wildtype[0]==predic[0] and wildtype[1]!=predic[1]:

                                    # Détermine s'il s'agit d'une élongation ou d'une troncation
                                    if predic[1]>wildtype[1]:
                                        fomonet_wt_vs_fomonet_mute = 'elongation'
                                    elif predic[1]<wildtype[1]:
                                        fomonet_wt_vs_fomonet_mute = 'troncation'
                            
                                # Le CDS prédit termine au même endroit que le CDS wild-type
                                elif wildtype[1]==predic[1] and wildtype[0]!=predic[0]:

                                    # Détermine s'il s'agit d'une élongation ou d'une troncation
                                    if predic[0]>wildtype[0]:
                                        fomonet_wt_vs_fomonet_mute = 'troncation'
                                    elif predic[0]<wildtype[0]:
                                        fomonet_wt_vs_fomonet_mute = 'elongation'

                                else:
                                    # Aucune position en commun avec le CDS wild-type
                                    if wildtype[0]!=predic[0] and wildtype[1]!=predic[1]:
                                        if (predic[0]-wildtype[0]) % 3 == 0 and (predic[1]-wildtype[1]) % 3 == 0:
                                            fomonet_wt_vs_fomonet_mute = 'inframe'
                                        else:
                                            fomonet_wt_vs_fomonet_mute = 'novel CDS'


                            elif isinstance(predic[1], str): # La position de fin tombe dans la mutation
                                position = predic[1].split('+')
                                predic_fin = int(position[0]) # Stockage de la dernière position faisant partie du transcrit de référence

                                # J'ai seulement confiance en la position de départ, car la position de fin n'est pas vraiment la bonne
                                # Les CDS ont la même position de départ
                                if predic[0]==wildtype[0]: 

                                    # Je peux déterminer s'il s'agit d'une troncation ou d'une élongation
                                    if predic_fin>=wildtype[1]:
                                        fomonet_wt_vs_fomonet_mute = 'elongation'
                                    elif predic_fin<wildtype[1]:
                                        fomonet_wt_vs_fomonet_mute = 'troncation'

                                # Si le départ n'est pas le même
                                else:
                                    # Je dois trouver si c'est inframe ou frameshift -> J'ai seulement confiance en la position de départ
                                    if (predic[0]-wildtype[0]) % 3 == 0:
                                        fomonet_wt_vs_fomonet_mute = 'inframe'
                                    else: 
                                        fomonet_wt_vs_fomonet_mute = 'novel CDS'

                        elif isinstance(predic[0], str): # Si la position de départ tombe dans la mutation
                            if isinstance(predic[1], int): # La position de fin ne tombe pas dans la mutation
                                position = predic[0].split('+')
                                predic_debut = int(position[0])

                                # Je ne peux pas avoir confiance en la position de départ
                                # Les CDS termine à la même position
                                if predic[1]==wildtype[1]:

                                    # Je peux déterminer s'il s'agit d'une troncation ou d'une élongation
                                    if predic_debut>=wildtype[0]:
                                        fomonet_wt_vs_fomonet_mute = 'troncation'
                                    elif predic_debut<wildtype[0]:
                                        fomonet_wt_vs_fomonet_mute = 'elongation'

                                # Les CDS n'ont pas la même fin
                                else:
                                    # Je dois trouver si c'est inframe ou frameshift -> J'ai seulement confiance en la position de fin
                                    if (predic[1]-wildtype[1]) % 3 == 0:
                                        fomonet_wt_vs_fomonet_mute = 'inframe'
                                    else: 
                                        fomonet_wt_vs_fomonet_mute = 'novel CDS'

                            elif isinstance(predic[1], str): # Dans le cas où les deux positions tombent dans la mutation
                                # Le CDS prédit est entièrement situé dans, par exemple une longue insertion. Il s'agit d'une séquence complètement nouvelle
                                fomonet_wt_vs_fomonet_mute = 'novel CDS intra-mutation'

            # Après la mutation, FOMOnet prédit plus de 1 CDS pour le transcrit muté
            elif len(prediction_mute)>len(fomonet_wt):
                fomonet_wt_vs_fomonet_mute = 'Gain de CDS : ' # FOMOnet prédit un gain de CDS

        
                for wildtype in fomonet_wt: # Itération sur le tuple du CDS canonique
                    nombre_iteration = 0 # Indique le nombre d'itération autour de prediction_mute. Chaque fois qu'on passe à un prochain, il va augmenter de 1
                    for predic in prediction_mute: # Itération sur les tuples contenus dans prediction_mute
                        nombre_iteration = nombre_iteration + 1 # Nouvelle itération = +1
                        
                        if nombre_iteration > 1: # Quand le nombre d'itération est supérieur à 1. 
                            fomonet_wt_vs_fomonet_mute += ' + ' # Ajoute un + entre les éléments

                    
                        if isinstance(predic[0], int): # La position de départ est un int (c'est-à-dire, pas dans la mutation)
                            if isinstance(predic[1], int): # La position de fin est un int (c'est-à-dire, pas dans la mutation)

                                # Je peux avoir confiance en la position de départ et en la position de fin

                                # Le CDS wild-type correspond au CDS prédit après mutation
                                if wildtype[0]==predic[0] and wildtype[1]==predic[1]:
                                    fomonet_wt_vs_fomonet_mute += 'normal'

                                # Le CDS prédit commence au même endroit que le CDS canonique
                                elif wildtype[0]==predic[0] and wildtype[1]!=predic[1]:

                                    # Détermine s'il s'agit d'une élongation ou d'une troncation
                                    if predic[1]>wildtype[1]:
                                        fomonet_wt_vs_fomonet_mute += 'elongation'
                                    elif predic[1]<wildtype[1]:
                                        fomonet_wt_vs_fomonet_mute += 'troncation'
                            
                                # Le CDS prédit termine au même endroit que le CDS canonique
                                elif wildtype[1]==predic[1] and wildtype[0]!=predic[0]:

                                    # Détermine s'il s'agit d'une élongation ou d'une troncation
                                    if predic[0]>wildtype[0]:
                                        fomonet_wt_vs_fomonet_mute += 'troncation'
                                    elif predic[0]<wildtype[0]:
                                        fomonet_wt_vs_fomonet_mute += 'elongation'

                                else:
                                    # Aucune position en commun avec le CDS canonique
                                    if wildtype[0]!=predic[0] and wildtype[1]!=predic[1]:
                                        if (predic[0]-wildtype[0]) % 3 == 0 and (predic[1]-wildtype[1]) % 3 == 0:
                                            fomonet_wt_vs_fomonet_mute += 'inframe'
                                        else:
                                            fomonet_wt_vs_fomonet_mute += 'novel CDS'


                            elif isinstance(predic[1], str): # La position de fin tombe dans la mutation
                                position = predic[1].split('+')
                                predic_fin = int(position[0]) # Stockage de la dernière position faisant partie du transcrit de référence

                                # J'ai seulement confiance en la position de départ, car la position de fin n'est pas vraiment la bonne
                                # Les CDS ont la même position de départ
                                if predic[0]==wildtype[0]: 

                                    # Je peux déterminer s'il s'agit d'une troncation ou d'une élongation
                                    if predic_fin>=wildtype[1]:
                                        fomonet_wt_vs_fomonet_mute += 'elongation'
                                    elif predic_fin<wildtype[1]:
                                        fomonet_wt_vs_fomonet_mute += 'troncation'

                                # Si le départ n'est pas le même
                                else:
                                    # Je dois trouver si c'est inframe ou frameshift -> J'ai seulement confiance en la position de départ
                                    if (predic[0]-wildtype[0]) % 3 == 0:
                                        fomonet_wt_vs_fomonet_mute += 'inframe'
                                    else: 
                                        fomonet_wt_vs_fomonet_mute += 'novel CDS'

                        elif isinstance(predic[0], str): # Si la position de départ tombe dans la mutation
                            if isinstance(predic[1], int): # La position de fin ne tombe pas dans la mutation
                                position = predic[0].split('+')
                                predic_debut = int(position[0])

                                # Je ne peux pas avoir confiance en la position de départ
                                # Les CDS termine à la même position
                                if predic[1]==wildtype[1]:

                                    # Je peux déterminer s'il s'agit d'une troncation ou d'une élongation
                                    if predic_debut>=wildtype[0]:
                                        fomonet_wt_vs_fomonet_mute += 'troncation'
                                    elif predic_debut<wildtype[0]:
                                        fomonet_wt_vs_fomonet_mute += 'elongation'

                                # Les CDS n'ont pas la même fin
                                else:
                                    # Je dois trouver si c'est inframe ou frameshift -> J'ai seulement confiance en la position de fin
                                    if (predic[1]-wildtype[1]) % 3 == 0:
                                        fomonet_wt_vs_fomonet_mute += 'inframe'
                                    else: 
                                        fomonet_wt_vs_fomonet_mute += 'novel CDS'

                            elif isinstance(predic[1], str): # Dans le cas où les deux positions tombent dans la mutation
                                # Le CDS prédit est entièrement situé dans, par exemple une longue insertion. Il s'agit d'une séquence complètement nouvelle
                                fomonet_wt_vs_fomonet_mute += 'novel CDS intra-mutation'

                    
        # Finalement, si le transcrit n'a pas de CDS canonique
        elif len(fomonet_wt)==0:

            # FOMOnet ne prédit aucun CDS
            if len(fomonet_wt)==len(prediction_mute): 
                fomonet_wt_vs_fomonet_mute = 'normal'

            # FOMOnet prédit un CDS: il s'agit d'un gain de CDS    
            elif len(fomonet_wt)<len(prediction_mute):
                x = len(prediction_mute) # Stock le nombre de CDS prédit par FOMOnet
                fomonet_wt_vs_fomonet_mute = f"Gain de CDS ({x})"
        

        elif len(fomonet_wt)>=2:
            fomonet_wt_vs_fomonet_mute = 'not coded yet in python :)'

    return fomonet_wt_vs_fomonet_mute



In [12]:
##### ÉTAPE 8: CRÉER LE FICHIER DES ANALYSES DES DONNÉES PRÉDITES PAR FOMONET avec les mutations splice#####
import os

# Création du dossier
folder_path = '../result'
# Création d'un nouveau dossier dans le dossier de destination
new_folder = os.path.join(folder_path, f'post_fomonet_{gene}')
if not os.path.exists(new_folder): # Si le dossier n'existe pas, il faut en créer un autre
    os.makedirs(new_folder)


fichier = f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_avec_splice.tsv'
with open(fichier, 'w') as f:
    f.write('')

with open(fichier, 'a') as file:
    file.write('TRANSCRIT_MUTE' + '\t' + 'CDS_CANONIQUE' + '\t' + 'FOMONET_WT' + '\t' + 'MUTE_NOT_CORRIGE' + '\t' + 'MUTE_CORRIGE' + '\t' + 'MUTATION_TYPE' + '\t' + 'MUTATION_CATEGORY'+ '\t' + 'CANONIQUE_VS_MUTE' + '\t' + 'FOMONET_WT_VS_MUTE' + '\t' + 'POSITION_DEBUT' + '\t' + 'POSITION_FIN' + '\t' + 'NUCLEOTIDE_AJOUTE' + '\t' + 'NUCLEOTIDE_DELETE' + '\t' + 'IS_MUTATION_IN_CDS_CANONIQUE' + '\t' + 'IS_MUTATION_IN_FOMONET_WT'  + '\n')
    for key in orf_corrige:
        info = key.split('_')
        transcrit = info[1]
        cds_canonique = get_cds_canonique(transcrit)
        fomonet_wt = get_fomonet_wt(transcrit)
        mute_corrige = orf_corrige[key]
        mute_not_corrige = orf_sans_stop[key]
        mutation_type = get_mutation_type(key)
        mutation_cat = get_mutation_cat(key)
        canonique_vs_mute = get_comparaison_canonical_vs_fomonet_mute(key)
        fomonet_wt_vs_fomonet_mute = get_comparaison_fomonet_wt_vs_fomonet_mute(key)
        debut = get_position_debut(key)
        fin = get_position_fin(key)
        ajoute = get_whats_add(key)
        delete = get_whats_delete(key)
        dans_canonique = is_mutation_in_cds_canonique(key)
        dans_fomonet_wt = is_mutation_in_fomonet_wt(key)
        #codon_start_canonique = get_codon_start_canonique(transcrit)
        #codon_start_wt = get_codon_start_fomonet_wt(transcrit)
        #codon_start_mute = get_codon_start_mute(key)
            

        # Écriture dans le fichier
        file.write(key + '\t' + str(cds_canonique) + '\t' + str(fomonet_wt) + '\t' + str(mute_not_corrige) + '\t' + str(mute_corrige) + '\t' + str(mutation_type) + '\t' + str(mutation_cat)  + '\t' + str(canonique_vs_mute) + '\t' + str(fomonet_wt_vs_fomonet_mute) + '\t' + str(debut) + '\t' + str(fin) + '\t' + str(ajoute) + '\t' + str(delete) + '\t' + str(dans_canonique) + '\t' + str(dans_fomonet_wt) + '\n')

In [13]:
# Vérification
# Il faut que le nombre de ligne retrouvé dans le fichier produit corresponde au nombre de données traité par FOMOnet (transcrit muté seulement)

# Je compte les lignes retrouvées dans le fichier.
count_line = 0
with open(f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_avec_splice.tsv', 'r') as file:
    for line in file:
        count_line += 1
        
if count_line-1 == len(orf_corrige): # J'enlève la ligne qui sert de 'header'
    print('Correct!')
else:
    print('Error')


Correct!


In [ ]:
# J'ai décidé de retirer les mutations 'splice'.

fichier = f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_sans_splice.tsv'
with open(fichier, 'w') as f:
    f.write('')

with open(f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_avec_splice.tsv', 'r') as file:
    for line in file:
        colonne = line.split('\t')
        if 'splice' not in colonne[6]:
            with open(f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_sans_splice.tsv', 'a') as f:
                f.write(line)
                

In [ ]:
# Vérification
# Je compte le nombre de ligne dans le premier fichier d'analyse
count_line = 0
with open(f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_avec_splice.tsv', 'r') as file:
    for line in file:
        count_line += 1
# Dans ce même fichier, je compte combien de splice on retrouve    
splice_count = 0
with open(f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_avec_splice.tsv', 'r') as file:
    for line in file:
        colonne = line.split('\t')
        if 'splice' in colonne[6]:
            splice_count += 1
# Dans le nouveau fichier sans splice, je compte combien il y a de ligne.
count_line_sans_splice = 0
with open(f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_sans_splice.tsv', 'r') as f:
    for line in f:
        count_line_sans_splice += 1
# Le nombre de ligne doit être identique au nombre de lignes initiales - le nombre de splice retrouvé
if (count_line - splice_count) == count_line_sans_splice:
    print('Correct!')
else:
    print('Error')
    
# Double vérification
categorie_mutation = set()
with open(f'../result/post_fomonet_{gene}/{gene}_analyse_fomonet_sans_splice.tsv', 'r') as f:
    for line in f:
        colonne = line.split('\t')
        cat_mut = colonne[6]
        categorie_mutation.add(cat_mut)

error_found = False # Marquage qui sert à arrêter dès qu'on trouve une erreur
for item in categorie_mutation: # Itération sur chaque item
    if 'splice' in item: # Si le nom splice se retrouve dans l'item du set()
        print('Error') # Cela imprime: Error
        error_found = True
        break  # On arrête la boucle dès qu'on trouve un 'splice'
    
# Si error_found reste toujours False
if not error_found:
    print('Double Correct!')


Correct!
Double Correct!
